Part II : Model Creation and Prediction

Team Name : sindhu & thapasya 

User names: sindhukc , mthapasya

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Kaggle Challenge

/content/drive/MyDrive/Kaggle Challenge


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Kaggle Challenge')

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [5]:
train_data = pd.read_csv('training_data.csv')
print(train_data.shape)
train_data.head()

(3141, 144)


,county,cases,deaths,date_index_converted,county_data_length,core_jaccard,core_cosine,core_intersection,social_jaccard,politics_jaccard,...,labor_force_rate,unemployment_rate,median_housing_cost,median_household_earnings,median_worker_earnings,percent_insured,percent_married,poverty_rate,median_property_value,percent_white
0,Richland,0,0,day_10,363,0.000000,0.00000,0.000000,0.000011,0.000151,...,55.5,7.5,675,41877,23210,90.5,48.3,15.6,103700,0.868085
1,Lawrence,119,0,day_105,256,0.000000,0.00000,0.000000,0.000000,0.000000,...,53.5,6.5,655,42874,23510,92.2,49.3,18.6,101500,0.954027
2,Wayne,49,0,day_90,769,0.000076,0.00037,0.006502,0.000039,0.000000,...,64.0,4.0,762,50383,26658,87.2,55.1,13.0,140100,0.950541
3,Fayette,7,0,day_85,36,0.000000,0.00000,0.000000,0.000000,0.000000,...,59.3,6.3,732,40503,25858,91.3,51.8,17.7,108900,0.940054
4,Trumbull,0,0,day_7,554,0.000000,0.00000,0.000000,0.000046,0.000000,...,56.4,5.9,661,43073,25800,91.7,49.1,17.2,101600,0.885724


# Feature Extraction and Data Processing

In [6]:
def convert_DI(data):
  return int(data[4:])

In [9]:
from sklearn.base import BaseEstimator
class Data_Processing(BaseEstimator):
  def __init__(self):
        pass
  def fit(self,dataset,y=None):
    return self
  
  def transform(self,dataset):
    #extracted number of days
    dataset['DIC_num']=dataset['date_index_converted'].apply(convert_DI)
    #calculated the death percentage
    dataset['death_percent']= (dataset['deaths']/dataset['total_pop'])*100
    #calculated the awarness percentage
    dataset['awarness_percent']= (dataset['county_data_length']/dataset['total_pop'])*100
    dataset= dataset.drop('date_index_converted',axis='columns')
    #one hot encoding for catgeorical variable county
    ct = ColumnTransformer([('one_hot_encoder', preprocessing.OneHotEncoder(), [0])], remainder='passthrough')
    data_encoded = ct.fit_transform(dataset)
    # Standard scaling the entire datset
    sc_X = StandardScaler(with_mean=False)
    df = sc_X.fit_transform(data_encoded)
    return df

In [10]:
X_train = train_data.drop("cases", axis='columns')
Y_train = train_data["cases"]
X_train.head()

,county,deaths,date_index_converted,county_data_length,core_jaccard,core_cosine,core_intersection,social_jaccard,politics_jaccard,politics_democratic_love_jaccard,...,labor_force_rate,unemployment_rate,median_housing_cost,median_household_earnings,median_worker_earnings,percent_insured,percent_married,poverty_rate,median_property_value,percent_white
0,Richland,0,day_10,363,0.000000,0.00000,0.000000,0.000011,0.000151,0.000053,...,55.5,7.5,675,41877,23210,90.5,48.3,15.6,103700,0.868085
1,Lawrence,0,day_105,256,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,53.5,6.5,655,42874,23510,92.2,49.3,18.6,101500,0.954027
2,Wayne,0,day_90,769,0.000076,0.00037,0.006502,0.000039,0.000000,0.000000,...,64.0,4.0,762,50383,26658,87.2,55.1,13.0,140100,0.950541
3,Fayette,0,day_85,36,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,59.3,6.3,732,40503,25858,91.3,51.8,17.7,108900,0.940054
4,Trumbull,0,day_7,554,0.000000,0.00000,0.000000,0.000046,0.000000,0.000000,...,56.4,5.9,661,43073,25800,91.7,49.1,17.2,101600,0.885724


In [11]:
X_train.dtypes[X_train.dtypes != 'int64'][X_train.dtypes != 'float64']

county                  object
date_index_converted    object
dtype: object

In [12]:
Y_train.head()

0      0
1    119
2     49
3      7
4      0
Name: cases, dtype: int64

In [13]:
X_train.head()

,county,deaths,date_index_converted,county_data_length,core_jaccard,core_cosine,core_intersection,social_jaccard,politics_jaccard,politics_democratic_love_jaccard,...,labor_force_rate,unemployment_rate,median_housing_cost,median_household_earnings,median_worker_earnings,percent_insured,percent_married,poverty_rate,median_property_value,percent_white
0,Richland,0,day_10,363,0.000000,0.00000,0.000000,0.000011,0.000151,0.000053,...,55.5,7.5,675,41877,23210,90.5,48.3,15.6,103700,0.868085
1,Lawrence,0,day_105,256,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,53.5,6.5,655,42874,23510,92.2,49.3,18.6,101500,0.954027
2,Wayne,0,day_90,769,0.000076,0.00037,0.006502,0.000039,0.000000,0.000000,...,64.0,4.0,762,50383,26658,87.2,55.1,13.0,140100,0.950541
3,Fayette,0,day_85,36,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,59.3,6.3,732,40503,25858,91.3,51.8,17.7,108900,0.940054
4,Trumbull,0,day_7,554,0.000000,0.00000,0.000000,0.000046,0.000000,0.000000,...,56.4,5.9,661,43073,25800,91.7,49.1,17.2,101600,0.885724


In [14]:
test_data = pd.read_csv('test_data.csv')
print(test_data.shape)
test_data.head()

(7331, 144)


,index,county,deaths,date_index_converted,county_data_length,core_jaccard,core_cosine,core_intersection,social_jaccard,politics_jaccard,...,labor_force_rate,unemployment_rate,median_housing_cost,median_household_earnings,median_worker_earnings,percent_insured,percent_married,poverty_rate,median_property_value,percent_white
0,0,Carroll,0,day_22,40,0.000000,0.000000,0.000000,0.000106,0.000000,...,58.2,4.7,713,47075,26589,88.5,56.9,14.8,116700,0.974052
1,1,Richland,3,day_118,17,0.000661,0.002567,0.058824,0.000494,0.000000,...,55.5,7.5,675,41877,23210,90.5,48.3,15.6,103700,0.868085
2,2,Portage,0,day_51,1210,0.000000,0.000000,0.000000,0.000013,0.000009,...,65.5,7.2,896,52552,25904,93.5,46.0,14.5,152000,0.909878
3,3,Auglaize,0,day_27,38,0.000000,0.000000,0.000000,0.000000,0.000000,...,67.0,3.4,758,54274,30438,95.7,59.4,9.0,143100,0.970007
4,4,Montgomery,0,day_8,4328,0.000005,0.000036,0.000462,0.000043,0.000033,...,61.8,8.0,830,43829,26292,92.0,44.8,17.9,112100,0.731997


In [16]:
#extract only few columns
col=['county', 'deaths', 'date_index_converted', 'county_data_length','total_pop','percent_25_34','percent_highschool','labor_force_rate',
     'unemployment_rate', 'median_housing_cost','median_household_earnings', 'median_worker_earnings','percent_insured', 'percent_married', 'poverty_rate','social_cosine',
     'entertainment_cosine','sports_cosine',
     'race_cosine','economy_cosine','foreign_cosine','religion_cosine','health_cosine',
     'health_technology_cosine','domestic_cosine','illness_cosine',
     'education_cosine','gender_cosine','median_property_value', 'percent_white']
X_train_data= X_train[col]
X_test =test_data[col]

In [17]:
#Created the pipeline to run the process across train and test data
model_pipeline = Pipeline(steps=[('Data Processing', Data_Processing())])
transformed_X=model_pipeline.fit_transform(X_train_data)
transformed_test=model_pipeline.fit_transform(X_test)
print(transformed_X.shape)
print(transformed_test.shape)

(3141, 119)
(7331, 119)


# Model Creation

LGBMRegressor with GridSearchCV

In [23]:
from lightgbm import LGBMRegressor
lgbm_params = {
    'learning_rate': [0.25,0.5],
    'n_estimators': [800,1000,1500],
    'num_leaves': [30],
    'max_depth': [None,5,10,15],
    'objective': ['regression_l2'],
    'reg_lambda': [0.1]
}
scores = []

lgbm_grid_search = GridSearchCV(LGBMRegressor(), lgbm_params, cv=5, scoring='r2', n_jobs=-1)
lgbm_grid_search.fit(transformed_X, Y_train)
print("Best LightGBM parameters: ", lgbm_grid_search.best_params_)
best_lgbm = lgbm_grid_search.best_estimator_
# {'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 800, 'num_leaves': 30, 'objective': 'regression_l2', 'reg_lambda': 0.1}
#last used     'learning_rate': [0.01, 0.1,0.5,1], 'n_estimators': [100,200,500,800],'num_leaves': [10,30, 50],'max_depth': [None,5,10,15],
    # 'objective': ['regression_l2'], 'reg_lambda': [0.1,0.5,0.7]
#new {'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 1500, 'num_leaves': 30, 'objective': 'regression_l2', 'reg_lambda': 0.1

Best LightGBM parameters:  {'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 1500, 'num_leaves': 30, 'objective': 'regression_l2', 'reg_lambda': 0.1}


AdaBoostRegressor with GridSearchCV

In [36]:

DTR=DecisionTreeRegressor(max_depth=15)
from sklearn.ensemble import AdaBoostRegressor
model_params_ada = {
   'n_estimators': [500,1000,1500], 
            'base_estimator':[DTR],
            'learning_rate': [0.25,0.5]
}

ada_grid_search = GridSearchCV(AdaBoostRegressor(random_state=32), model_params_ada, cv=5, scoring='r2', n_jobs=-1)
ada_grid_search.fit(transformed_X, Y_train)
print("Best ADABoost parameters: ", ada_grid_search.best_params_)
best_ada_dt = ada_grid_search.best_estimator_


Best ADABoost parameters:  {'base_estimator': DecisionTreeRegressor(max_depth=15), 'learning_rate': 0.25, 'n_estimators': 500}


LightGBM + AdaBoost +Decision Tree

In [37]:
from sklearn.ensemble import VotingRegressor
new_model_ada = VotingRegressor(
    estimators=[
        # ('random_forest', best_RF),
        # ('DT', best_dt),
        ('LightGBM', best_lgbm),
        ('AdaBoost', best_ada_dt),
        # ('xgboost', best_XGB)
    ])
new_model_ada.fit(transformed_X, Y_train)
scr_ada = cross_validate(new_model_ada, transformed_X, Y_train, scoring=['r2'])
y_pred_ada = new_model_ada.predict(transformed_test)
r2_score(Y_train, new_model_ada.predict(transformed_X))

0.9999994789854594

In [39]:
scr_ada

{'fit_time': array([35.65133595, 34.04481864, 36.43315148, 35.47287035, 35.56333685]),
 'score_time': array([0.17205739, 0.17493176, 0.17048407, 0.28242564, 0.16527057]),
 'test_r2': array([0.85191875, 0.94766138, 0.94554776, 0.95751331, 0.96624876])}

In [40]:
y_pred_ada

array([ -2.39686793, 244.0433997 , -12.61867503, ...,  -2.32830224,
       193.58365062, -27.82253571])

In [41]:
y_predXGB_v2=[]
for value in y_pred_ada:
  if value < 0:
    y_predXGB_v2.append(0)
  else:
    y_predXGB_v2.append(int(value))
results= pd.DataFrame(y_predXGB_v2,columns=['Cases'])
results.to_csv('results.csv')

Results: We achieved R2 value of 0.86239 with the above ensemble model. We used the Light GBM Regressor model and AdaBoost Regressor with Decesion Tree. 

# **Other Model Used For Prediction**

Other Models that were used to predict the cases although the R2 obtained was lower than 0.86239:
1.   XGB Regressor with GridSearcghCV to get the best Estimator
2.   Random Forest with GridSearcghCV to get the best Estimator

XGB Regressor:
We got the below parameters as the best parameter for XGB Regressor, although it did not give better prediction when used with other models.

{'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 500}

We used GridSearchCV to obtain the best estimator for our training data.

In [ ]:
# XGBR_params = {
#     'XGBRegressor' : {
#         'model': XGBRegressor(random_state=32),
#         'params': {
#                 'learning_rate': [0.1,0.5],
#                 'n_estimators': [500,800],
#                 'max_depth': [5,7],
#                 'gamma': [0.1,0.3],
#         }
#     }
# }
# scores = []

# for model_name, mp in XGBR_params.items():
#     clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
#     clf.fit(transformed_X, Y_train,)
#     scores.append({
#         'model': model_name,
#         'best_score': clf.best_score_,
#         'best_params': clf.best_params_,
#         'best_estimator':clf.best_estimator_
#     })
    
# df_model_XGBR = pd.DataFrame(scores,columns=['model','best_score','best_params','best_estimator'])
# df_model_XGBR

Randome Forest:
We got the below parameters as the best parameter for Random Forest, although it did not give better prediction when used with other models.

{'ccp_alpha': 0.01, 'max_depth': None, 'n_estimators': 1000}

We used GridSearchCV to obtain the best estimator for our training data.

In [ ]:
# model_params_rf = {
#     'random_forest': {
#         'model': RandomForestRegressor(random_state=32),
#         'params' : {
#             'n_estimators': [1000,1500], 
#             'max_depth': [None ,1], 
#             'ccp_alpha':[0.05,0.01]
#         }
#     },   
# }

# scores = []
# # best_RF

# for model_name, mp in model_params_rf.items():
#     clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
#     clf.fit(transformed_X, Y_train,)
#     scores.append({
#         'model': model_name,
#         'best_score': clf.best_score_,
#         'best_params': clf.best_params_,
#         'best_estimator':clf.best_estimator_
#     })
    
# df_model_rf = pd.DataFrame(scores,columns=['model','best_score','best_params','best_estimator'])
# df_model_rf

We used Voting Regressor to fit the base regressor of all tghe models on the training dataset and obtain the average of each prediction to form the final prediction.

Below code shows the implementation of best estimator of Random Forest, Light GBM and XGB.

In [ ]:
# from sklearn.ensemble import VotingRegressor
# new_model = VotingRegressor(
#     estimators=[
#         ('random_forest', best_RF),
#         ('LightGBM', best_lgbm),
#         #  ('CatBoost', best_cb),
#         ('xgboost', best_XGB)
#     ])
# new_model.fit(transformed_X, Y_train)
# scr_RF_XGB_LGBM = cross_validate(new_model, transformed_X, Y_train, scoring=['r2'])
# y_pred_RF_XGB_LGBM = new_model.predict(transformed_test)
# r2_score(Y_train, new_model.predict(transformed_X))

Decision Tree : We got the below parameters as the best parameter for Decision Tree, although it did not give better prediction when used with other models.

 {'max_depth': 12, 'max_leaf_nodes': 150, 'min_samples_leaf': 2, 'min_samples_split': 2}

We used GridSearchCV to obtain the best estimator for our training data.

In [ ]:
# from sklearn.tree import DecisionTreeRegressor
# dt_params2 = {
#               "min_samples_split": [2],
#               "max_depth": [10,12,15],
#               "min_samples_leaf": [1,2],
#               "max_leaf_nodes": [100,150],
# }
# scores = []

# dt_grid_search = GridSearchCV(DecisionTreeRegressor(), dt_params2, cv=5, scoring='r2', n_jobs=-1)
# dt_grid_search.fit(transformed_X, Y_train)
# print("Best LightGBM parameters: ", dt_grid_search.best_params_)
# best_dt = dt_grid_search.best_estimator_
# {'max_depth': 12, 'max_leaf_nodes': 150, 'min_samples_leaf': 2, 'min_samples_split': 2}
#  last used:  "min_samples_split": [2,5,10, 20, 40],"max_depth": [2, 6, 8,10,12,15],"min_samples_leaf": [2,4,5,10,20, 40],
#  "max_leaf_nodes": [5, 20, 100,150,200,250],

#best :Best LightGBM parameters:  {'max_depth': 15, 'max_leaf_nodes': 100, 'min_samples_leaf': 1, 'min_samples_split': 2}